The purpose of my bot is to tweet out whenever John Paul Stevens gives a speech that is posted on SCOTUS website at https://www.supremecourt.gov/publicinfo/speeches/speeches.aspx. This would include the subject of the speech along with a link that directs the audience to a pdf of the actual speech. The twitter bot scrapes accurately and gives the correct information that a person would want to know concerning what the speech is and where to go to view it. It also tweets as it is intended to for every instance of an update on the speech page.

The google sheet can be found at https://docs.google.com/spreadsheets/d/1Nr66RqtUg-1fP6K8C2OgVJWIU195E1Xja0ZqWZKQlus/edit#gid=0 , the twitter bot is found at https://twitter.com/bcrockett3, and the 

Adapted from the notebook found at [How to Build a Law Bot](https://lawyerist.com/how-build-law-bot/)

## Install libraries

If you haven't already, you may need to install some dependencies. On the command line, run the following to install/update gspread, oauth2client, PyOpenSSL, and python-twitter.
```
pip install gspread
pip install --upgrade oauth2client
pip install PyOpenSSL
pip install python-twitter
```
Library installs are one and done. So after doing this once, you should be all set. 

## Import modules and set variables

Now we're getting into the bot's code. This is what will run every time your bot is called. 

You will need to create a new Google Sheet (same instructions as [last time](https://lawyerist.com/126074/online-forms-meet-local-document-automation-cut-and-paste-coding/)). Delete rows 2-999. This is because the code below appends values to the end of your sheet. So if you fail to remove rows 2-999, values will be appended to row 1000. Additionally, it looks at the last row of the sheet for your old values. So right off the bat it will be looking at your one solitary row. Also, delete columns D through Z to avoind having to print a bunch of empty columns.

As for a Twitter account and Twitter credentials, follow the instruction in [this post](https://lawyerist.com/?p=127093). 

In [4]:
# Load the module for visiting and reading websites.
import urllib.request
# Load the module for running regular expressions (regex).
import re 
# Load the module for date and time stuff.
import datetime
# Define the variable now as equal to the current date and time.
now = datetime.datetime.now()

In [5]:
# Set the URL you want to scrape.
url_1 = "https://www.supremecourt.gov/publicinfo/speeches/speeches.aspx"

# If you want to scrape data from multiple pages, you can, 
# just replicate the above and below but change url_1 to url_2 et al.

In [6]:
# Load the module for accessing Google Sheets.
import gspread
# Load the module needed for securely communicating with Google Sheets.
from oauth2client.service_account import ServiceAccountCredentials
# The scope for your access credentials
scope = ['https://spreadsheets.google.com/feeds']

# Your spreadsheet's ID
document_key = "1Nr66RqtUg-1fP6K8C2OgVJWIU195E1Xja0ZqWZKQlus" 
#              ^^^^^^^^^^^ SWAP OUT FOR YOUR DOCUMENT ID/KEY
# Your Google project's .json key
credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../../../key.json', scope)
#                                                                              ^^^^^^^^ SWAP OUT FOR YOUR JSON KEY
# Use your credentials to authorize yourself.
gc = gspread.authorize(credentials)
# Open up the Sheet with the defined ID.
wks = gc.open_by_key(document_key)

#########################################
#
#  NOTE: The name of the sheet you are 
#  trying to access should be in the 
#  parenthetical below (e.g., Data). By
#  Default this is probably "Sheet1".
#
#########################################
worksheet = wks.worksheet("Sheet1")

# Count the number of rows in your Sheet &
# resize to remove blank rows.
worksheet.resize(worksheet.row_count)

In [7]:
# Print out the old values stored in your sheet 
# Note: The first time you run this code, it will be empty as nothing has yet to be stored in your sheet.

print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################

['0', '0', '0', '0']


In [8]:
# Import the relevant Twitter libraries so you can use Twitter.
import twitter
from twitter import TwitterError

# create the following four text files and add them to the same diretctry as you 
# Google API key. In each file add the appropriate value found when retrieving your 
# Twitter credentials

with open('../../../../../key.txt', 'r') as myfile:
    key=myfile.read()
    
with open('../../../../../secret.txt', 'r') as myfile:
    secret=myfile.read()
    
with open('../../../../../token_key.txt', 'r') as myfile:
    token_key=myfile.read()

with open('../../../../../token_secret.txt', 'r') as myfile:
    token_secret=myfile.read()

# Set you Twitter API credentials.
api = twitter.Api(consumer_key=key,
                  consumer_secret=secret,
                  access_token_key=token_key,
                  access_token_secret=token_secret)

## Read the contents of your first webpage

When you run the next cell, your program will visit the first URL you defined above. It will then print out that page's HTML. 

In [9]:
p_1 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_1).read()
print(p_1)

b'\r\n\r\n<!DOCTYPE html>\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="ctl00_ctl00_Head1"><meta http-equiv="X-UA-Compatible" content="IE=edge" /><meta http-equiv="content-type" content="txt/html; charset=utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1" />\r\n        <script type="text/javascript" src="/js/jquery-3.1.0.min.js"></script>\r\n        <script type="text/javascript" src="/js/bootstrap.js"></script>\r\n        <link rel="stylesheet" type="text/css" href="/css/font-awesome.min.css" />\r\n        <link rel="Stylesheet" type="text/css" href="/css/bootstrap.min.css" />\r\n        <link rel="Stylesheet" type="text/css" href="/css/bootstrap-theme.min.css" />\r\n        <link rel="stylesheet" type="text/css" href="/styles/newBootStrap2.css" />\r\n    \r\n\r\n<!-- HTML5 shim and Respond.js IE8 support of HTML5 elements and media queries -->\r\n   \r\n        <!--[if lt IE 9]>\r\n          <script src="/js/html5shiv.js"></script>\r\n     

# Two Data Points, One Match

---------------------------
## Parse the site's contents

In [10]:
res_1 = re.search(b"John Paul Stevens.*\s*.*href='(.*\.pdf).*'>(.*)</a>",p_1)
output_1 = res_1.group(1).decode('UTF-8')
print(output_1)
output_2 = res_1.group(2).decode('UTF-8')
print(output_2)

JPS_Speech_Arlington_County_Bar_Association_06-10-2016.pdf
Arlington County Bar Association, Arlington, VA, June 10, 2016


## Post to Twitter and Save to Google (Two Data Point, One Match)

In [11]:
if (res_1 and (worksheet.row_values(worksheet.row_count)[1]) != output_1
          and (worksheet.row_values(worksheet.row_count)[2]) != output_2):
    # same as above but now comparing two values
    
    try:
        # Post to Twitter.
        status = api.PostUpdate('%s https://www.supremecourt.gov/publicinfo/speeches/%s'%(output_2,output_1))
        print(status.text)
    except TwitterError:
        # Post to Twitter.
        status = api.PostUpdate('%s new speech https://www.supremecourt.gov/publicinfo/speeches/%s'%(output_2,output_1))
        print(status.text)

    # Save to Google only after Tweeting
    worksheet.append_row([now,output_1,output_2])

Arlington County Bar Association, Arlington, VA, June 10, 2016 https://t.co/uJY1XrK2Zr
